In [1]:
# !pip install transformers

In [2]:
import os, gc, sys, time, collections, random
import numpy as np
import pandas as pd

from typing import Dict, Optional, Union, Any, List, Tuple

from pathlib import Path
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn

import torch.utils.data as D
from torch.utils.data.dataset import Dataset, IterableDataset
from torch.utils.data.dataloader import DataLoader

from transformers import DistilBertTokenizerFast
from transformers import DistilBertModel
from transformers import BertTokenizerFast
from transformers import BertModel
from transformers import Trainer
from transformers import AutoModel, AutoTokenizer
from transformers.data.processors.utils import InputFeatures
from transformers import TrainingArguments
from transformers.trainer_utils import EvalLoopOutput
from transformers.trainer import logging
from transformers.file_utils import is_torch_tpu_available, is_sagemaker_mp_enabled
from transformers.trainer_pt_utils import find_batch_size, nested_concat, nested_numpify, nested_truncate, nested_detach

### Folders and Dataframes

In [3]:
DATA_PATH = Path('/home/commonlit/data/')
assert DATA_PATH.exists()
MODELS_PATH = Path('/home/commonlit/models/')
if not MODELS_PATH.exists():
    os.mkdir(MODELS_PATH)
assert MODELS_PATH.exists()

In [4]:
!ls {DATA_PATH}

commonlitreadabilityprize.zip  test.csv        train.csv
sample_submission.csv	       train-orig.csv


In [5]:
train_df = pd.read_csv(DATA_PATH/'train.csv')
test_df = pd.read_csv(DATA_PATH/'test.csv')
sample_df = pd.read_csv(DATA_PATH/'sample_submission.csv')

In [6]:
train_df

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
1,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
2,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
3,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
4,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007
...,...,...,...,...,...,...
2834,25ca8f498,https://sites.ehe.osu.edu/beyondpenguins/files...,CC BY-SA 3.0,When you think of dinosaurs and where they liv...,1.711390,0.646900
2835,2c26db523,https://en.wikibooks.org/wiki/Wikijunior:The_E...,CC BY-SA 3.0,So what is a solid? Solids are usually hard be...,0.189476,0.535648
2836,cd19e2350,https://en.wikibooks.org/wiki/Wikijunior:The_E...,CC BY-SA 3.0,The second state of matter we will discuss is ...,0.255209,0.483866
2837,15e2e9e7a,https://en.wikibooks.org/wiki/Geometry_for_Ele...,CC BY-SA 3.0,Solids are shapes that you can actually touch....,-0.215279,0.514128


In [7]:
train_df[train_df['id'] == '5127fb10f']['excerpt'].values

array(['The Battle of Waterloo was a battle that was fought mostly between French and British forces. Napoleon was crowned as Emperor of France in 1804. Then he launched many successful attacks on other countries in Europe. France soon had an empire that stretched from Spain to the Russian border. The only country that was still not captured was Great Britain. The Royal Navy had many ships, so invasion by France was not possible. However, Great Britain was not strong enough to stop Napoleon and his army from taking over most of mainland Europe.\nNapoleon seemed unstoppable until two separate campaigns caused his empire to fall apart. He gathered a huge army to invade and conquer Russia once and for all in 1812. However, he did not think that he would have very many difficulties and it turned out he did. His army was caught by the Russian winter and destroyed by the weather and lack of food.',
       'Napoleon was crowned as Emperor of France in 1804, and then launched the successful Na

In [8]:
train_df[train_df['id'] == '5127fb10f']['target']

255   -0.338548
256   -0.338548
Name: target, dtype: float64

### Prepare Cross Validation

In [9]:
target = train_df['target'].to_numpy()

In [10]:
num_bins = int(np.floor(np.log2(len(train_df))) + 1)
train_df.loc[:, 'bins'] = pd.cut(train_df['target'], bins=num_bins, labels=False)

In [11]:
train_df[['target', 'bins']].groupby(['bins']).agg(['mean', 'count'])

target      
          mean count
bins                
0    -3.413097    43
1    -2.969369    79
2    -2.526589   172
3    -2.106393   269
4    -1.652726   366
5    -1.201150   419
6    -0.748874   482
7    -0.309570   408
8     0.130016   312
9     0.560407   183
10    0.978923    83
11    1.399764    23

In [12]:
kf = StratifiedKFold(n_splits=num_bins)

In [13]:
for i, (t_, v_) in enumerate(kf.split(X=train_df, y=train_df.bins.values)):
    train_df.loc[v_, 'kfold'] = i

In [14]:
train_df['kfold'] = train_df['kfold'].astype(np.uint8)

In [15]:
train_df = train_df.drop('bins', axis=1)

In [16]:
train_df

,id,url_legal,license,excerpt,target,standard_error,kfold
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009,0
1,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009,0
2,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805,0
3,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676,0
4,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007,0
...,...,...,...,...,...,...,...
2834,25ca8f498,https://sites.ehe.osu.edu/beyondpenguins/files...,CC BY-SA 3.0,When you think of dinosaurs and where they liv...,1.711390,0.646900,11
2835,2c26db523,https://en.wikibooks.org/wiki/Wikijunior:The_E...,CC BY-SA 3.0,So what is a solid? Solids are usually hard be...,0.189476,0.535648,11
2836,cd19e2350,https://en.wikibooks.org/wiki/Wikijunior:The_E...,CC BY-SA 3.0,The second state of matter we will discuss is ...,0.255209,0.483866,11
2837,15e2e9e7a,https://en.wikibooks.org/wiki/Geometry_for_Ele...,CC BY-SA 3.0,Solids are shapes that you can actually touch....,-0.215279,0.514128,11


In [17]:
bin_list = list(range(num_bins))
random.shuffle(bin_list)
bin_list

[2, 3, 1, 8, 5, 7, 0, 4, 6, 10, 11, 9]

### Metrics

In [18]:
def rmse_score(y_true, y_pred):
    return np.sqrt(np.mean((y_pred - y_true) ** 2))

def rmse_score_2(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [19]:
a = np.random.rand(10)
b = np.random.rand(10)

In [20]:
rmse_score(a, b), rmse_score_2(a, b)

(0.33664903124978596, 0.33664903124978596)

### Configuration

In [21]:
class CONFIG():
    model_name = 'distilroberta'
    batch_size = 128
    max_len = 256
    save_dir = f'trained/{model_name}'
    num_workers = 2
    epochs = 30
    pretrained_transformers_model = f'{model_name}-base'

In [22]:
cfg = CONFIG()

### Prepare train test split

In [23]:
def create_split(fold = [1]):
    valid_df = train_df[train_df['kfold'].isin(fold)]
    valid_text = valid_df['excerpt'].values
    valid_target = valid_df['target'].values
    training_df = train_df[~train_df['kfold'].isin(fold)]
    train_text = training_df['excerpt'].values
    train_target = training_df['target'].values
    return train_text, train_target, valid_text, valid_target

In [24]:
train_text, train_target, valid_text, valid_target = create_split([0])
len(train_text), len(valid_text)

(2602, 237)

### Prepare Tokenizers

In [25]:
tokenizer = AutoTokenizer.from_pretrained(cfg.pretrained_transformers_model)
# Save the tokenizer so that you can download the files and move it to a Kaggle dataset.
tokenizer.save_pretrained(cfg.save_dir)

('trained/distilroberta/tokenizer_config.json',
 'trained/distilroberta/special_tokens_map.json',
 'trained/distilroberta/vocab.json',
 'trained/distilroberta/merges.txt',
 'trained/distilroberta/added_tokens.json',
 'trained/distilroberta/tokenizer.json')

In [26]:
encoded_dict = tokenizer(train_df['excerpt'].values[0],
                                return_tensors='pt',
                                max_length=cfg.max_len,
                                padding='max_length',
                                truncation=True)
decoded = tokenizer.decode(encoded_dict["input_ids"].squeeze())
decoded

'<s>When the young people returned to the ballroom, it presented a decidedly changed appearance. Instead of an interior scene, it was a winter landscape.\nThe floor was covered with snow-white canvas, not laid on smoothly, but rumpled over bumps and hillocks, like a real snow field. The numerous palms and evergreens that had decorated the room, were powdered with flour and strewn with tufts of cotton, like snow. Also diamond dust had been lightly sprinkled on them, and glittering crystal icicles hung from the branches.\nAt each end of the room, on the wall, hung a beautiful bear-skin rug.\nThese rugs were for prizes, one for the girls and one for the boys. And this was the game.\nThe girls were gathered at one end of the room and the boys at the other, and one end was called the North Pole, and the other the South Pole. Each player was given a small flag which they were to plant on reaching the Pole.\nThis would have been an easy matter, but each traveller was obliged to wear snowshoes

In [27]:
encoded_dict['input_ids'].shape

torch.Size([1, 256])

In [28]:
def convert_to_list(t):
    return t.flatten().long()

class CommonLitDataset(nn.Module):
    def __init__(self, text, target, tokenizer, max_len=128):
        self.excerpt = text
        self.target = target
        self.max_len = max_len
        self.tokenizer = tokenizer
    
    def __getitem__(self, idx):
        encode = self.tokenizer(self.excerpt[idx],
                                return_tensors='pt',
                                max_length=self.max_len,
                                padding='max_length',
                                truncation=True)
        return InputFeatures(input_ids=convert_to_list(encode['input_ids']),
                      attention_mask=convert_to_list(encode['attention_mask']),
                      label=torch.tensor(self.target[idx]))
    
    def __len__(self):
        return len(self.excerpt)

In [29]:
def create_train_valid_ds(tokenizer, train_text, train_target, valid_text, valid_target):
    train_ds = CommonLitDataset(train_text, train_target, tokenizer, cfg.max_len)
    valid_ds = CommonLitDataset(valid_text, valid_target, tokenizer, cfg.max_len)
    return train_ds, valid_ds

In [30]:
# train_dl = D.DataLoader(train_ds, batch_size=cfg.batch_size, shuffle=True, num_workers=cfg.num_workers)
# train_dl = D.DataLoader(valid_ds, batch_size=cfg.batch_size, shuffle=False, num_workers=cfg.num_workers)

In [31]:
# encode, target = next(iter(train_dl))

In [32]:
# encode.keys(), target.shape, encode['input_ids'].shape, encode['attention_mask'].shape

In [33]:
# encode['input_ids'][0].squeeze()

### Model

In [34]:
# You can use a Transformer model of your choice.
# transformer_model = DistilBertModel.from_pretrained(cfg.pretrained_transformers_model)
transformer_model = AutoModel.from_pretrained(cfg.pretrained_transformers_model)

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [35]:
# transformer_out = transformer_model(input_ids=encode['input_ids'].squeeze(), attention_mask=encode['attention_mask'].squeeze())

In [36]:
# dict(transformer_out)['last_hidden_state'].shape

In [37]:
# torch.mean(transformer_out.last_hidden_state, axis=1).shape

In [38]:
# sample_layer = nn.Linear(768, 1)

In [39]:
from torch.nn import functional as F

In [40]:
from transformers import PreTrainedModel

class CommonLitModel(PreTrainedModel):
    def __init__(self):
        super(PreTrainedModel, self).__init__()
        self.transformer_model = AutoModel.from_pretrained(cfg.pretrained_transformers_model)
        self.drop = nn.Dropout(0.5)
        self.layer_norm = nn.LayerNorm(768)
        self.out = nn.Linear(768, 1)
        self.config = self.transformer_model.config
        self._init_weights(self.layer_norm)
        
    def _init_weights(self, module):
        if isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def forward(self, input_ids, attention_mask):
        transformer_out = self.transformer_model(input_ids=input_ids.squeeze(), attention_mask=attention_mask.squeeze(), output_hidden_states=False)
        x = transformer_out.pooler_output
#         x = transformer_out.last_hidden_state[:, 0, :] Distilbert
        x = self.layer_norm(x)
        x = self.drop(x)
        x = self.out(x)
        return x
    
    def floating_point_ops(self, inputs: Dict[str, Union[torch.Tensor, Any]]):
        """
        For models that inherit from :class:`~transformers.PreTrainedModel`, uses that method to compute the number of
        floating point operations for every backward + forward pass. If using another model, either implement such a
        method in the model or subclass and override this method.
        Args:
            inputs (:obj:`Dict[str, Union[torch.Tensor, Any]]`):
                The inputs and targets of the model.
        Returns:
            :obj:`int`: The number of floating-point operations.
        """
        return 0

In [41]:
model = CommonLitModel()

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [42]:
encoded_dict.input_ids.shape

torch.Size([1, 256])

In [43]:
transformer_model = transformer_model.cuda()
sample_out = transformer_model(encoded_dict.input_ids.cuda(), encoded_dict.attention_mask.cuda(), output_hidden_states=True)

In [44]:
train_ds, valid_ds = create_train_valid_ds(tokenizer, train_text, train_target, valid_text, valid_target)

In [45]:
encode = train_ds[0]

In [46]:
encode.attention_mask.unsqueeze(0).shape, encoded_dict.input_ids.shape

(torch.Size([1, 256]), torch.Size([1, 256]))

In [47]:
sample_out = transformer_model(encode.input_ids.unsqueeze(0).cuda(), encode.attention_mask.unsqueeze(0).cuda())

In [48]:
sample_out.pooler_output.shape

torch.Size([1, 768])

### Training

In [49]:
import wandb

In [50]:
loss_fct = nn.MSELoss()

In [51]:
def create_training_args(fold):
    training_args = TrainingArguments(
        output_dir=str(MODELS_PATH/f'{cfg.model_name}-{fold}'),
        overwrite_output_dir=True,
        do_train=True,
        do_eval=True,
        per_device_train_batch_size=cfg.batch_size,
        per_device_eval_batch_size=cfg.batch_size,
        num_train_epochs=cfg.epochs,
        logging_strategy="epoch",
        logging_first_step=True,
        save_steps=40000,
        fp16=True,
        evaluation_strategy="epoch",
        save_total_limit = 3,
        load_best_model_at_end=True,
        metric_for_best_model='mse',
        greater_is_better=False,
        gradient_accumulation_steps=1,
        learning_rate=5e-5
    )
    return training_args

In [52]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    return {'mse': mean_squared_error(logits, labels), 'rmse': rmse_score_2(logits, labels)}

In [53]:
tokenizer = AutoTokenizer.from_pretrained(cfg.pretrained_transformers_model)

In [54]:
logger = logging.get_logger(__name__)

class CommonLitTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        input_ids = inputs.pop("input_ids")
        attention_mask = inputs.pop("attention_mask")
        outputs = model(input_ids, attention_mask)
        logits = outputs
        loss = loss_fct(logits.flatten(),
                        labels.float().flatten())
        zero_cat = torch.zeros([1, 1]).to(outputs.device)
        return (loss, torch.cat([zero_cat, outputs])) if return_outputs else loss

In [55]:
!rm -rf /home/commonlit/models/bert-*

In [56]:
%%time

from transformers import EarlyStoppingCallback

bin_step = 1
bestmodels = []
eval_rmses = []
for i in range(0, num_bins, bin_step):
    train_bins = bin_list[i:i+bin_step]
    print('train_bins', f'{i}: {train_bins}')
    tokenizer = AutoTokenizer.from_pretrained(cfg.pretrained_transformers_model)
    train_text, train_target, valid_text, valid_target = create_split([i])
    train_ds, valid_ds = create_train_valid_ds(tokenizer, train_text, train_target, valid_text, valid_target)
    training_args = create_training_args(i)
    model = CommonLitModel()
    wandb.init(project=f"commonlit_2_{cfg.model_name}")
    trainer = CommonLitTrainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=valid_ds,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=9)]
    )
    trainer.train()
    trainer.save_model()
    print('training_args.output_dir', training_args.output_dir)
    tokenizer.save_pretrained(training_args.output_dir)
    result = trainer.evaluate()
    bestmodels.append(trainer.state.best_model_checkpoint)
    print('best_model_checkpoint', trainer.state.best_model_checkpoint)
    print('result', result)
    eval_rmses.append(result['eval_rmse'])

train_bins 0: [2]


Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
wandb: Currently logged in as: gilf (use `wandb login --relogin` to force relogin)


Epoch,Training Loss,Validation Loss,Mse,Rmse
1,1.449800,0.927750,0.927750,0.963198
2,0.690700,0.359931,0.359931,0.599943
3,0.521700,0.624031,0.624031,0.789956
4,0.402300,0.644199,0.644199,0.802620
5,0.386300,0.312032,0.312032,0.558598
6,0.277100,0.979583,0.979583,0.989739
7,0.412400,0.781236,0.781236,0.883876
8,0.254600,0.346870,0.346870,0.588957
9,0.219700,0.335746,0.335746,0.579436
10,0.248000,0.399956,0.399956,0.632421


training_args.output_dir /home/commonlit/models/distilroberta-0


best_model_checkpoint /home/commonlit/models/distilroberta-0/checkpoint-252
result {'eval_loss': 0.29492491483688354, 'eval_mse': 0.29492491483688354, 'eval_rmse': 0.5430698990821838, 'eval_runtime': 0.7974, 'eval_samples_per_second': 297.204, 'epoch': 21.0, 'eval_mem_cpu_alloc_delta': 0, 'eval_mem_gpu_alloc_delta': 0, 'eval_mem_cpu_peaked_delta': 0, 'eval_mem_gpu_peaked_delta': 1510867456}
train_bins 1: [3]


...

train/loss,0.1235
train/learning_rate,3e-05
train/epoch,12.0
train/global_step,252
_runtime,180
_timestamp,1622468694
_step,26
eval/loss,0.28209
eval/mse,0.28209
eval/rmse,0.53112
eval/runtime,0.7907


train/loss,█▅▃▂▂▂▂▁▁▁▁▁▁
train/learning_rate,█▇▇▆▆▅▅▄▃▃▂▂▁
train/epoch,▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▆▆▆▆▇▇▇▇████
train/global_step,▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▆▆▆▆▇▇▇▇████
_runtime,▁▁▁▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▇▇▇▇████
_timestamp,▁▁▁▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▇▇▇▇████
_step,▁▁▂▂▂▂▃▃▃▃▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇██
eval/loss,▇▁▁█▂▅▆▂▆▁▇▅▁
eval/mse,▇▁▁█▂▅▆▂▆▁▇▅▁
eval/rmse,▇▂▁█▃▅▇▂▇▁▇▅▁
eval/runtime,▁▁▂▁▂▁▂▁▁█▁▁▃


Epoch,Training Loss,Validation Loss,Mse,Rmse
1,1.308300,0.471911,0.471911,0.686958
2,0.614900,0.362517,0.362517,0.602094
3,0.545200,0.305048,0.305048,0.552311
4,0.488100,0.583896,0.583896,0.764131
5,0.325000,0.384538,0.384538,0.620111
6,0.292800,0.826335,0.826335,0.909029
7,0.279000,0.586251,0.586251,0.765670
8,0.217500,0.324985,0.324985,0.570075
9,0.227200,0.476744,0.476744,0.690466
10,0.169500,0.301841,0.301841,0.549401


training_args.output_dir /home/commonlit/models/distilroberta-11


best_model_checkpoint /home/commonlit/models/distilroberta-11/checkpoint-210
result {'eval_loss': 0.3018411099910736, 'eval_mse': 0.3018410801887512, 'eval_rmse': 0.5494006276130676, 'eval_runtime': 0.8052, 'eval_samples_per_second': 293.079, 'epoch': 19.0, 'eval_mem_cpu_alloc_delta': 0, 'eval_mem_gpu_alloc_delta': 0, 'eval_mem_cpu_peaked_delta': 0, 'eval_mem_gpu_peaked_delta': 1510867456}
CPU times: user 58min 7s, sys: 15min 10s, total: 1h 13min 17s
Wall time: 45min 1s


In [57]:
BEST_MODEL_FOLDER = MODELS_PATH/cfg.model_name/'best'
!rm -rf {BEST_MODEL_FOLDER}
!mkdir -p {BEST_MODEL_FOLDER}

In [67]:
from shutil import copyfile

for i, best_model in enumerate(bestmodels):
    best_model_file = f'{best_model}/pytorch_model.bin'
    if Path(best_model_file).exists():
        copyfile(best_model_file, f'{BEST_MODEL_FOLDER}/{i}_pytorch_model.bin')
        tokenizer_path = Path(BEST_MODEL_FOLDER/f'tokenizer-{i}')
        tokenizer_path.mkdir(parents=True, exist_ok=True)
        assert tokenizer_path.exists()

        tokenizer_json = Path(f'{MODELS_PATH/cfg.model_name}-{i}/tokenizer.json')
        assert tokenizer_json.exists()
        copyfile(tokenizer_json, tokenizer_path/'tokenizer.json')

        vocab_txt = Path(f'{MODELS_PATH/cfg.model_name}-{i}/vocab.json')
        assert vocab_txt.exists()
        copyfile(vocab_txt, tokenizer_path/'vocab.json')

        config_json = Path(f'{MODELS_PATH/cfg.model_name}-{i}/config.json')
        assert config_json.exists()
        copyfile(config_json, tokenizer_path/'config.json')
    else:
        print(f'{best_model_file} is missing')

In [68]:
import shutil

shutil.make_archive(MODELS_PATH/cfg.model_name/'best_models', 'zip', BEST_MODEL_FOLDER)

'/home/commonlit/models/distilroberta/best_models.zip'

In [60]:
'Mean best RSME losses', np.array(eval_rmses).mean()

('Mean best RSME losses', 0.526772141456604)

In [64]:
state_dict = torch.load(str(MODELS_PATH/f'distilroberta-0/checkpoint-252/pytorch_model.bin'))

In [65]:
loaded_model = CommonLitModel()

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [66]:
loaded_model.load_state_dict(state_dict)

<All keys matched successfully>